In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)
    print('gpu')

gpu


In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size =100


In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/' ,
                          train= True, 
                          transform = transforms.ToTensor(), 
                          download = False)
mnist_test = dsets.MNIST(root= 'MNIST_data/', 
                         train= False , 
                         transform = transforms.ToTensor(), 
                         download = False)

/workspace/NIA/NIA_AI_DATASET_2021-ST-GCAE/.local/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                          batch_size= batch_size, 
                                          shuffle = True, 
                                          drop_last = True)


In [6]:
linear1= torch.nn.Linear(784, 256 , bias =True)
linear2= torch.nn.Linear(256, 256 , bias = True)
linear3 = torch.nn.Linear(256, 10, bias = True)
relu= torch.nn.ReLU()

In [7]:
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)


Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [8]:
model = torch.nn.Sequential(linear1 ,relu , linear2, relu, linear3).to(device)

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


In [10]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost =0
    for X,Y in data_loader:
        X= X.view(-1,28*28).to(device)
        Y= Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost +=cost / total_batch
    print('Epoch: ', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost))
print('Learning finished')        

Epoch:  0001 cost =  128.848510742
Epoch:  0002 cost =  35.990943909
Epoch:  0003 cost =  23.008337021
Epoch:  0004 cost =  16.110439301
Epoch:  0005 cost =  11.650862694
Epoch:  0006 cost =  8.514619827
Epoch:  0007 cost =  6.413101673
Epoch:  0008 cost =  4.843065739
Epoch:  0009 cost =  3.679252625
Epoch:  0010 cost =  2.772321224
Epoch:  0011 cost =  2.180697918
Epoch:  0012 cost =  1.674829721
Epoch:  0013 cost =  1.252921343
Epoch:  0014 cost =  1.002785444
Epoch:  0015 cost =  0.850125968
Learning finished


In [11]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1,28*28).to(device)
    Y_test = mnist_test.test_labels().to(device)
    
    prediction = model(X)
    correct_prediction = torch.argmax(prediction,1) ==Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())
    
    r=random.randint(0,len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r+1].view(-1, 28*28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r+1].to(device)
    
    print('Label ' ,Y_single_data )
    single_prediction = model(X_single_data)
    print('Prediction: ',torch.argmax(single_prediction, 1).item())

/workspace/NIA/NIA_AI_DATASET_2021-ST-GCAE/.local/lib/python3.8/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/workspace/NIA/NIA_AI_DATASET_2021-ST-GCAE/.local/lib/python3.8/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


TypeError: 'Tensor' object is not callable